In [1]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
import tensorflow as tf

In [2]:
mnist = read_data_sets("./new_dataset", one_hot=True, reshape=False, validation_size=0)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./new_dataset/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./new_dataset/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./new_dataset/t10k-images-idx3-ubyte.gz
Extracting ./new_dataset/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
CKPT_DIR = 'new_ckpt_dir_3'

In [4]:
class Network3(object):
    def __init__(self):
        self.global_step = tf.Variable(0,trainable = False)
        self.x = tf.placeholder(tf.float32, [None, 28, 28, 1], name = "x_input")
        self.label = tf.placeholder(tf.float32, [None, 10], name = "y_input")
        self.lr = 0.003
        self.W1 = tf.Variable(tf.truncated_normal([5, 5, 1, 4], stddev=0.1))   # 5x5 patch, 1 input channel, K output channels
        self.B1 = tf.Variable(tf.ones([4])/10)
        self.W2 = tf.Variable(tf.truncated_normal([5, 5, 4, 8], stddev=0.1))
        self.B2 = tf.Variable(tf.ones([8])/10)
        self.W3 = tf.Variable(tf.truncated_normal([4, 4, 8, 12], stddev=0.1))
        self.B3 = tf.Variable(tf.ones([12])/10)
        self.W4 = tf.Variable(tf.truncated_normal([7 * 7 * 12, 200], stddev=0.1))
        self.B4 = tf.Variable(tf.ones([200])/10)
        self.W5 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1))
        self.B5 = tf.Variable(tf.ones([10])/10)
        stride = 1  # output is 28x28
        self.Y1 = tf.nn.relu(tf.nn.conv2d(self.x, self.W1, strides=[1, stride, stride, 1], padding='SAME') + self.B1)
        stride = 2  # output is 14x14
        self.Y2 = tf.nn.relu(tf.nn.conv2d(self.Y1, self.W2, strides=[1, stride, stride, 1], padding='SAME') + self.B2)
        stride = 2  # output is 7x7
        self.Y3 = tf.nn.relu(tf.nn.conv2d(self.Y2, self.W3, strides=[1, stride, stride, 1], padding='SAME') + self.B3)
        # reshape the output from the third convolution for the fully connected layer
        self.YY = tf.reshape(self.Y3, shape=[-1, 7 * 7 * 12])
        self.Y4 = tf.nn.relu(tf.matmul(self.YY, self.W4) + self.B4)
        self.Ylogits = tf.matmul(self.Y4, self.W5) + self.B5
        self.predict = tf.nn.softmax(self.Ylogits, name = "z_output")
        self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = self.Ylogits, labels = self.label)
        self.loss = tf.reduce_mean(self.cross_entropy)*100
        # accuracy of the trained model, between 0 (worst) and 1 (best)
        self.predict = tf.equal(tf.argmax(self.predict, 1), tf.argmax(self.label, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.predict, tf.float32))
        self.train = tf.train.AdamOptimizer(self.lr).minimize(self.cross_entropy, global_step = self.global_step)

In [5]:
class Train(object):
    
    def __init__(self):
        self.net = Network3()
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def train(self):
        batch_size = 64
        train_step = 10000
        step = 0
        #Save Model every 1000 steps
        save_interval = 1000
        saver = tf.train.Saver(max_to_keep = 10)
        ckpt = tf.train.get_checkpoint_state(CKPT_DIR)
        if ckpt and ckpt.get_checkpoint_state(CKPT_DIR):
            saver.restore(self.sess,ckpt.model_checkpoint_path)
            step = self.sess.run(self.net.global_step)
            print('continue from')
            print('  -> Minibatch update : ',step)
        while step < train_step:
            x, label = mnist.train.next_batch(batch_size)
            _,loss = self.sess.run([self.net.train,self.net.loss], feed_dict = {self.net.x: x, self.net.label:label})
            step = self.sess.run(self.net.global_step)
            if step % 1000 == 0:
                print('No %6d step, current loss: %.3f'%(step,loss))
            if step % save_interval == 0:
                saver.save(self.sess,CKPT_DIR + '/model',global_step = step)
    
    def calculate_accuracy(self):
        test_x = self.data.test.images
        test_label = self.data.test.labels
        acc = self.sess.run(self.net.accuracy,feed_dict = {self.net.x:test_x,self.net.label:test_label})
        print("acc: %.3f, Number of pic test: %d " % (acc, len(test_label)))

In [6]:
if __name__ == '__main__':
    model = Train()
    model.train()
#     model.calculate_accuracy()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

No   1000 step, current loss: 12.968
No   2000 step, current loss: 0.396
No   3000 step, current loss: 4.282
No   4000 step, current loss: 5.561
No   5000 step, current loss: 3.835
No   6000 step, current loss: 0.304
No   7000 step, current loss: 0.073
No   8000 step, current loss: 2.808
No   9000 step, current loss: 13.186
No  10000 step, current loss: 1.994
